In [ ]:
import talib
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(r'C:\Users\Deekshant\Desktop\FOX\backtesting\data\dummy.csv')
# data.drop('Unnamed: 0',axis=1,inplace=True)
data['date'] = pd.to_datetime(data['Gmt time'])
data.rename({'open':'Open','high':'High','low':'Low','close':'Close'},axis=1,inplace=True)
# data.set_index('date',inplace=True)

In [ ]:
data = data.resample('15T').agg({'Open': 'first',
                                 'High': 'max', 
                                 'Low': 'min', 
                                 'Close': 'last'})
data.reset_index(inplace=True)

In [ ]:
data['ema50'] = talib.EMA(data['Close'],50)
data['ema100'] = talib.EMA(data['Close'],100)
data['ema200'] = talib.EMA(data['Close'],200)

In [ ]:
def EMA(df, base, target, period, alpha=False):
    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
    if (alpha == True):
        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
    else:
        df[target] = con.ewm(span=period, adjust=False).mean()
    df[target].fillna(0, inplace=True)
    return df

def ATR(df, period, ohlc=['open', 'high', 'low', 'close']):
    atr = 'ATR_' + str(period)
    if not 'TR' in df.columns:
        df['h-l'] = df[ohlc[1]] - df[ohlc[2]]
        df['h-yc'] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
        df['l-yc'] = abs(df[ohlc[2]] - df[ohlc[3]].shift())
        df['TR'] = df[['h-l', 'h-yc', 'l-yc']].max(axis=1)
        df.drop(['h-l', 'h-yc', 'l-yc'], inplace=True, axis=1)
    EMA(df, 'TR', atr, period, alpha=True)
    return df

def SuperTrend(df, period, multiplier, ohlc=['Open', 'High', 'Low', 'Close']):
    ATR(df, period, ohlc=ohlc)
    atr = 'ATR_' + str(period)
    st = 'ST' #+ str(period) + '_' + str(multiplier)
    stx = 'STX' #  + str(period) + '_' + str(multiplier)

    # Compute basic upper and lower bands
    df['basic_ub'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
    df['basic_lb'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]

    # Compute final upper and lower bands
    df['final_ub'] = 0.00
    df['final_lb'] = 0.00
    for i in range(period, len(df)):
        df['final_ub'].iat[i] = df['basic_ub'].iat[i] if df['basic_ub'].iat[i] < df['final_ub'].iat[i - 1] or \
                                                         df[ohlc[3]].iat[i - 1] > df['final_ub'].iat[i - 1] else \
        df['final_ub'].iat[i - 1]
        df['final_lb'].iat[i] = df['basic_lb'].iat[i] if df['basic_lb'].iat[i] > df['final_lb'].iat[i - 1] or \
                                                         df[ohlc[3]].iat[i - 1] < df['final_lb'].iat[i - 1] else \
        df['final_lb'].iat[i - 1]

    # Set the Supertrend value
    df[st] = 0.00
    for i in range(period, len(df)):
        df[st].iat[i] = df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[
            i] <= df['final_ub'].iat[i] else \
            df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] > \
                                     df['final_ub'].iat[i] else \
                df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] >= \
                                         df['final_lb'].iat[i] else \
                    df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] < \
                                             df['final_lb'].iat[i] else 0.00

        # Mark the trend direction up/down
    df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), 'down', 'up'), np.NaN)

    # Remove basic and final bands from the columns
    df.drop(['basic_ub', 'basic_lb', 'final_ub', 'final_lb'], inplace=True, axis=1)

    df.fillna(0, inplace=True)
    return df

data = SuperTrend(data,7,3)

In [ ]:
data.drop(['TR','ATR_7'],axis=1,inplace=True)

In [ ]:
dataUp = ((data['ema50']>data['ema100']) & (data['ema100']>data['ema200']) & (data['STX']=='up'))
dataDown = ((data['ema50']<data['ema100']) & (data['ema100']<data['ema200']) & (data['STX']=='down'))

In [ ]:
data['Position'] = np.where(dataUp,1,np.where(dataDown,-1,0))
data['Change'] = data['Close'] - data['Close'].shift()
data['Performance'] = data['Position'].shift() * data['Change']

In [ ]:
price_data = data.copy()

In [ ]:
#price_data['Position_Change'] = price_data['Position'].diff()
price_data['Long_Position'] = np.where(price_data['Position'] == 1, 1, 0)
price_data['Short_Position'] = np.where(price_data['Position'] == -1, 1, 0)
price_data['Long_Position'] = price_data['Long_Position'].diff()
price_data['Short_Position'] = price_data['Short_Position'].diff()

In [ ]:
# Marking Entry prices for all eligible running positions (long)
price_data['Long_Open_Price'] = np.where(price_data['Long_Position'] == 1, price_data['Close'], np.where(price_data['Long_Position'] == -1, -np.inf, np.nan))
price_data['Long_Open_Price'] = price_data['Long_Open_Price'].shift()
price_data['Long_Open_Price'] = price_data['Long_Open_Price'].ffill()
price_data['Long_Open_Price'] = price_data['Long_Open_Price'].replace(-np.inf, np.nan)

# Marking Entry prices for all eligible running positions (short)
price_data['Short_Open_Price'] = np.where(price_data['Short_Position'] == 1, price_data['Close'], np.where(price_data['Short_Position'] == -1, -np.inf, np.nan))
price_data['Short_Open_Price'] = price_data['Short_Open_Price'].shift()
price_data['Short_Open_Price'] = price_data['Short_Open_Price'].ffill()
price_data['Short_Open_Price'] = price_data['Short_Open_Price'].replace(-np.inf, np.nan)

In [ ]:
# Setting Target and SL
# Target Points = 10 pips, SL = 5 pips
target = 0.0010
sl = 0.0005
price_data['Long_Target_Price'] = price_data['Long_Open_Price'] + target
price_data['Long_SL_Price'] = price_data['Long_Open_Price'] - sl


price_data['Short_Target_Price'] = price_data['Short_Open_Price'] - target
price_data['Short_SL_Price'] = price_data['Short_Open_Price'] + sl

In [ ]:
# See where all is Target met or SL breached (long)
price_data['Long_SL_Fired'] = np.where(price_data['Close'] < price_data['Long_SL_Price'], 1, 0)
price_data['Long_Target_Fired'] = np.where(price_data['Close'] > price_data['Long_Target_Price'], 1, 0)

# See where all is Target met or SL breached (short)
price_data['Short_SL_Fired'] = np.where(price_data['Close'] > price_data['Short_SL_Price'], 1, 0)
price_data['Short_Target_Fired'] = np.where(price_data['Close'] < price_data['Short_Target_Price'], 1, 0)

In [ ]:
# Track all close positions (We would later try to identify the first one among them for any entry)
price_data['Long_Updated_Position'] = price_data['Long_Position']
price_data['Long_Updated_Position'] = np.where((price_data['Long_SL_Fired'] == 1) | (price_data['Long_Target_Fired'] == 1), -1, price_data['Long_Updated_Position'])

# Set last point as compulsory exit
price_data['Long_Updated_Position'].iloc[-1] = -1


# Repeat the same for short
price_data['Short_Updated_Position'] = price_data['Short_Position']
price_data['Short_Updated_Position'] = np.where((price_data['Short_SL_Fired'] == 1) | (price_data['Short_Target_Fired'] == 1), -1, price_data['Short_Updated_Position'])

# Set last point as compulsory exit
price_data['Short_Updated_Position'].iloc[-1] = -1

In [ ]:
# Identifying the first close position time for any entry and marking others invalid
price_data['Long_Position_Working'] = price_data['Long_Updated_Position']
price_data['Long_Position_Working'].replace(0, np.nan, inplace=True)
price_data['Long_Position_Working'].ffill(inplace=True)
price_data['Long_Position_Working'] = price_data['Long_Position_Working'].diff()

# Valid closes would have the diff value as -2. (The one with Open just before --> Diff: (-1) - 1 = -2)
invalid_exits_index = (price_data['Long_Updated_Position'] == -1) & (price_data['Long_Position_Working'] != -2)
price_data['Long_Updated_Position'][invalid_exits_index] = 0


# Repeat the same for short
# Identifying the first close position time for any entry and marking others invalid
price_data['Short_Position_Working'] = price_data['Short_Updated_Position']
price_data['Short_Position_Working'].replace(0, np.nan, inplace=True)
price_data['Short_Position_Working'].ffill(inplace=True)
price_data['Short_Position_Working'] = price_data['Short_Position_Working'].diff()

# Valid closes would have the diff value as -2. (The one with Open just before --> Diff: (-1) - 1 = -2)
invalid_exits_index = (price_data['Short_Updated_Position'] == -1) & (price_data['Short_Position_Working'] != -2)
price_data['Short_Updated_Position'][invalid_exits_index] = 0

In [ ]:
# Update close prices
price_data['Long_Closed_Price'] = np.where(price_data['Long_Updated_Position'] == -1, price_data['Close'], np.nan)

# Was Target hit?
target_hit_index = (price_data['Long_Updated_Position'] == -1) & (price_data['Long_Target_Fired'] == 1)
price_data['Long_Closed_Price'][target_hit_index] = price_data['Long_Target_Price']

# Was SL hit?
sl_hit_index = (price_data['Long_Updated_Position'] == -1) & (price_data['Long_SL_Fired'] == 1)
price_data['Long_Closed_Price'][sl_hit_index] = price_data['Long_SL_Price']


## Repeat the same for Short
# Update close prices
price_data['Short_Closed_Price'] = np.where(price_data['Short_Updated_Position'] == -1, price_data['Close'], np.nan)

# Was Target hit?
target_hit_index = (price_data['Short_Updated_Position'] == -1) & (price_data['Short_Target_Fired'] == 1)
price_data['Short_Closed_Price'][target_hit_index] = price_data['Short_Target_Price']

# Was SL hit?
sl_hit_index = (price_data['Short_Updated_Position'] == -1) & (price_data['Short_SL_Fired'] == 1)
price_data['Short_Closed_Price'][sl_hit_index] = price_data['Short_SL_Price']

In [ ]:
price_data[price_data['Short_Updated_Position'] != 0].head()

In [ ]:
# Update PnL Column
price_data['PnL'] = np.where(price_data['Long_Updated_Position'] == -1, price_data['Long_Closed_Price'] - price_data['Long_Open_Price'], np.where(price_data['Short_Updated_Position'] == -1, - price_data['Short_Closed_Price'] + price_data['Short_Open_Price'], np.nan))


In [ ]:
# Total PnL
sum(price_data[price_data['PnL'].notna()]['PnL'])

In [ ]:
price_data[['Long_Open_Price', 'Long_Closed_Price', 'Short_Open_Price', 'Short_Closed_Price', 'PnL']][price_data['PnL'].notna()]

In [ ]:
len(price_data[price_data['PnL'].notna()])